In [1]:
import sys
from pathlib import Path

print("Python:", sys.executable)
print("CWD:", Path.cwd())



Python: /Users/macski/Projects/ai-multisystem-platform/.venv/bin/python
CWD: /Users/macski/Projects/ai-multisystem-platform/services/nhl_goal_scorers


In [3]:
list((PROJECT_ROOT / "services" / "nhl_goal_scorers" / "data" / "raw").glob("*"))[:20]


[]

In [4]:
import pandas as pd
from pathlib import Path

# We are already inside services/nhl_goal_scorers
BASE_DIR = Path.cwd()

CSV_PATH = BASE_DIR / "data" / "raw" / "moneypuck_skaters_2023_2024_regular.csv"

print("CWD:", BASE_DIR)
print("CSV:", CSV_PATH)
print("Exists:", CSV_PATH.exists())


CWD: /Users/macski/Projects/ai-multisystem-platform/services/nhl_goal_scorers
CSV: /Users/macski/Projects/ai-multisystem-platform/services/nhl_goal_scorers/data/raw/moneypuck_skaters_2023_2024_regular.csv
Exists: True


In [5]:
df = pd.read_csv(CSV_PATH)

print("Rows:", len(df))
print("Columns:", len(df.columns))

df.head(3)


Rows: 4620
Columns: 154


,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
0,8480950,2023,Ilya Lyubushkin,TOR,D,other,74,2881.0,56.0,5.08,...,18.00,21.61,195.0,191.0,0.11,0.02,3.0,1.0,3.0,1.0
1,8480950,2023,Ilya Lyubushkin,TOR,D,all,74,76034.0,1717.0,4.47,...,176.13,171.61,3226.0,3132.0,0.00,0.00,0.0,0.0,0.0,0.0
2,8480950,2023,Ilya Lyubushkin,TOR,D,5on5,74,61758.0,1389.0,4.47,...,107.09,104.80,2302.0,2303.0,6.54,0.64,140.0,11.0,108.0,8.0


In [6]:
# Check situation distribution
df["situation"].value_counts()


situation
other    924
all      924
5on5     924
4on5     924
5on4     924
Name: count, dtype: int64

In [7]:
df_all = df[df["situation"] == "all"].copy()

print("Rows (all):", len(df_all))
df_all.head(3)


Rows (all): 924


,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
1,8480950,2023,Ilya Lyubushkin,TOR,D,all,74,76034.0,1717.0,4.47,...,176.13,171.61,3226.0,3132.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8478438,2023,Tommy Novak,NSH,C,all,71,60495.0,1254.0,52.55,...,171.40,175.65,3236.0,3389.0,0.0,0.0,0.0,0.0,0.0,0.0
11,8476925,2023,Colton Sissons,NSH,C,all,81,80548.0,1859.0,35.27,...,192.16,163.13,3806.0,3363.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
BASE_COLS = [
    "playerId",
    "name",
    "team",
    "position",
    "games_played",
    "I_F_goals",
    "I_F_shotsOnGoal",
    "I_F_xGoals",
    "icetime"
]

df_base = df_all[BASE_COLS].copy()
df_base.head(3)


,playerId,name,team,position,games_played,I_F_goals,I_F_shotsOnGoal,I_F_xGoals,icetime
1,8480950,Ilya Lyubushkin,TOR,D,74,0.0,47.0,2.71,76034.0
6,8478438,Tommy Novak,NSH,C,71,18.0,113.0,13.95,60495.0
11,8476925,Colton Sissons,NSH,C,81,15.0,131.0,20.86,80548.0


In [9]:
df_base["goals_per_game"] = df_base["I_F_goals"] / df_base["games_played"]
df_base["shots_per_game"] = df_base["I_F_shotsOnGoal"] / df_base["games_played"]
df_base["shooting_pct"] = df_base["I_F_goals"] / df_base["I_F_shotsOnGoal"]

df_base.sort_values("goals_per_game", ascending=False).head(10)


,playerId,name,team,position,games_played,I_F_goals,I_F_shotsOnGoal,I_F_xGoals,icetime,goals_per_game,shots_per_game,shooting_pct
2966,8483432,Luca Del Bel Belluz,CBJ,C,1,1.0,2.0,0.17,575.0,1.000000,2.000000,0.500000
2861,8482787,Nikita Chibrikov,WPG,R,1,1.0,3.0,0.17,663.0,1.000000,3.000000,0.333333
2536,8483397,Marc McLaughlin,BOS,C,1,1.0,2.0,0.13,759.0,1.000000,2.000000,0.500000
3091,8479318,Auston Matthews,TOR,C,81,69.0,369.0,50.77,101897.0,0.851852,4.555556,0.186992
186,8477933,Sam Reinhart,FLA,C,82,57.0,233.0,44.25,99875.0,0.695122,2.841463,0.244635
741,8475786,Zach Hyman,EDM,L,80,54.0,290.0,52.51,93602.0,0.675000,3.625000,0.186207
2991,8477492,Nathan MacKinnon,COL,C,82,51.0,405.0,37.26,112246.0,0.621951,4.939024,0.125926
4096,8478864,Kirill Kaprizov,MIN,L,75,46.0,277.0,38.57,97096.0,0.613333,3.693333,0.166065
2281,8478550,Artemi Panarin,NYR,L,82,49.0,303.0,30.61,98985.0,0.597561,3.695122,0.161716
4036,8476887,Filip Forsberg,NSH,L,82,48.0,347.0,38.77,93015.0,0.585366,4.231707,0.138329


In [10]:
from pathlib import Path

# We are inside services/nhl_goal_scorers, so processed path is local to this folder
PROCESSED_DIR = Path.cwd() / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = PROCESSED_DIR / "player_goal_rates.csv"
df_base.to_csv(OUT_PATH, index=False)

print("Saved:", OUT_PATH)
print("File exists:", OUT_PATH.exists())


Saved: /Users/macski/Projects/ai-multisystem-platform/services/nhl_goal_scorers/data/processed/player_goal_rates.csv
File exists: True


In [11]:
import pandas as pd
pd.read_csv(OUT_PATH).head(5)


,playerId,name,team,position,games_played,I_F_goals,I_F_shotsOnGoal,I_F_xGoals,icetime,goals_per_game,shots_per_game,shooting_pct
0,8480950,Ilya Lyubushkin,TOR,D,74,0.0,47.0,2.71,76034.0,0.000000,0.635135,0.000000
1,8478438,Tommy Novak,NSH,C,71,18.0,113.0,13.95,60495.0,0.253521,1.591549,0.159292
2,8476925,Colton Sissons,NSH,C,81,15.0,131.0,20.86,80548.0,0.185185,1.617284,0.114504
3,8480860,Kevin Bahl,NJD,D,82,1.0,63.0,2.27,85591.0,0.012195,0.768293,0.015873
4,8478028,Fredrik Olofsson,COL,L,57,3.0,45.0,5.45,33372.0,0.052632,0.789474,0.066667
